In [1]:
import bluequbit
import numpy as np
from matplotlib import pyplot as plt

In [2]:
bq = bluequbit.init("Y11Q9TV91o98lDCVk1dCvc3UEDvq0Mpx")

[BQ-PYTHON-SDK][WARNING] - Beta version 0.12.0b1 of BlueQubit Python SDK is being used.


In [3]:
from qiskit import QuantumCircuit

In [4]:
qc = QuantumCircuit.from_qasm_file('P6_titan_pinnacle.qasm')

In [5]:
qc.measure_all()
print(qc.count_ops())

OrderedDict({'u3': 6992, 'cz': 3494, 'measure': 62, 'barrier': 1})


In [10]:
# Import required packages
from qiskit.transpiler import PassManager, CouplingMap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.passes import OptimizeSwapBeforeMeasure
from qiskit_ibm_runtime import QiskitRuntimeService
 
from qiskit_ibm_transpiler.ai.routing import AIRouting
from qiskit_ibm_transpiler.ai.synthesis import AILinearFunctionSynthesis
from qiskit_ibm_transpiler.ai.collection import CollectLinearFunctions
from qiskit_ibm_transpiler.ai.synthesis import AIPauliNetworkSynthesis
from qiskit_ibm_transpiler.ai.collection import CollectPauliNetworks

In [ ]:
QiskitRuntimeService.save_account(token='y', instance="ibm-q/open/main",
    channel="ibm_quantum",       # Specify the channel
    overwrite=True              # Overwrite existing credentials if any
)

In [ ]:
ibm_torino = QiskitRuntimeService().backend('ibm_sherbrooke')


In [ ]:
# Create a fully connected coupling map for 62 qubits
coupling_list = []
for i in range(62):
    for j in range(i + 1, 62):
        coupling_list.append([i, j])

coupling_map = CouplingMap(coupling_list)

ai_passmanager = PassManager([
  AIRouting(backend=ibm_torino, optimization_level=3, layout_mode="optimize", local_mode=True),  # Route circuit
  CollectLinearFunctions(),  # Collect Linear Function blocks
  AILinearFunctionSynthesis(backend=ibm_torino, local_mode=True),  # Re-synthesize Linear Function blocks
  CollectPauliNetworks(), # Collect Pauli Networks blocks
  AIPauliNetworkSynthesis(backend=ibm_torino, local_mode=False),  # Re-synthesize Pauli Network blocks. Only available in the Qiskit Transpiler Service
,AILinearFunctionSynthesis(backend=ibm_torino, max_threads=200)  # Re-synthesize Linear Function blocks using 20 threads max

])
transpiled_circuit = ai_passmanager.run(qc)



In [ ]:
result = bq.run(transpiled_circuit, device='quantum', shots=10000) 